## **Dependencies** 
###### (all of these packages should be added to the python path in the .bashrc)

Python 2 (several of the packages below are not compatible yet with python 3)

HEALPix (and dependencies) (from https://healpix.jpl.nasa.gov/)

Healpy (from pip installer)

PyFITS<=3.3 (from pip installer) (*NB: Flipper is **not** compatible with PyFITS 3.5 currently*)

tqdm (from pip installer)

Flipper (from https://github.com/sudeepdas/flipper) {this needs PyFits and astropy installed via pip}

FlipperPol (from https://github.com/msyriac/flipperPol)

PolSpice (from http://www2.iap.fr/users/hivon/software/PolSpice/)


*NB: There may be errors relating to the CardList() function when running Flipper depending on the version of PyFITS installed. If this persists, change the PyFITS version to 3.3 or replace CardList() with Header() in the Flipper LiteMap.py file.*

**For a full description of the code please refer to Philcox, Sherwin & van Engelen (MNRAS, 2018) or contact ohep2@alumni.cam.ac.uk**

# Basic Usage


## 1) Split Map into Tiles

The code base for this process was written by Alex van Engelen (CITA) and partitions a full-sky HEALPix TQU polarization map into tiles of a desired width. 

This is controlled via the python `tile_creation/cutoutmapsPhilcox.py` file. To select tile-sizes and load HEALPix maps a `.dict` dictionary should be created. A sample is given in `tile_creation/sample_dictionary.dict`.

The dictionary file contains several important parameters including the location of the HEALPix file and tile_widths and must be altered for each new test. The parameters are explained in the sample dictionary comments.

We note that the HEALPix map should be of high resolution (NSIDE=2048) and should be a dust-map (e.g. Vansyngel+ 2017 high-resolution map, Planck FFP10 etc.). Instrumental noise and lensing are simulated within the HADES code and added to cut-outs of this map later.

**NB:** to analyse a section of the sky (which is far less resource intensive) a HEALPix Mask file must be provided, upgraded to the same resolution as the HEALPix dust map. The mask used in the nominal HADES paper can be provided on request.

Now cut out the tiles using this script and the dictionary as follows. This may take a while depending on the processor speed and can generate several 10s GBs of files depending on the settings used.

```python -u ~/HADES/tile_creation/cutoutmapsPhilcox.py ~/HADES/tile_creation/sample_dictionary.dict```

Then make power maps from these tiles (this configures the mask files and also must be done for each new HEALPix map or tile configuration.

```python -u ~/HADES/tile_creation/powerMapsPhilcox.py ~/HADES/tile_creation/sample_dictionary.dict```

This completes the pre-configuration of map 'tiles'. The HEALPix map (with an appropriate mask) has now been partitioned into a number of small tiles of desired width. These are located in the directory specified in the edited dictionary file in the subdirectory `3deg1/` {if tiles of width 3deg separated by 1deg were chosen}. These files are later found and analysed by the `hades` code.

***A note on file structure***:
*The tile naming system has the following structure: *

*fvsmap{TYPE}_{NUMBER}.fits 

where TYPE specifies the Mask, T, Q or U polarizations and the NUMBER specifies the positions (with RA and Dec coordinates of the map centers given in the pickle files fvsmapDecs.pkl and fvsmapRAs.pkl).*

*NB: The prefix 'fvs' is irrelevant here but used throughout the code so should not be changed. *

## 2) Tile Analysis with HADES

## 2.1) Parameter Files

Essential (and non-essential) parameters for the analysis of a HEALPix map are controlled by a class in the `hades/params.py` file

In [6]:
# import the parameters
from hades.params import BICEP
import numpy as np
a=BICEP() # parameter class

Parameters are described in detail in the python file. The key parameters to alter are listed below:

In [7]:
a.hades_dir = '/home/oliver/HADES/' # directory where HADES is installed
a.root_dir = '/home/oliver/hades_testing/' # directory to house all simulation cut-outs + analysis products. This must be the directory housing the (e.g. 3deg3/) cut-outs.
a.flipU = True # This converts between COSMO [in many input maps] and IAU [used by flipper] polarisation conventions. If true, this reverses the sign of the U-polarisation map which has non-trivial effects on the analysis.
    
## Tile parameters
a.map_size = 3 # tile (cut-out section) width in degrees
a.sep = 3 # separation of the center of each tile. This may be set as less than the map_size to allow higher resolution plots with overlapping tiles but destroys the statistical independence

## MC parameters
a.N_sims = 500 # No. of MC simulations used to create parameter distributions
a.N_bias = 500 # No. simulations used for realisation-dependent debiasing

## Experimental parameters
a.freq = 150 # Desired map frequency in GHz. Currently only 150 GHz (peak BICEP sensitivity) and 353 GHz (original simulations) are implemented - [these rely on non-linear color conversions
a.f_dust = 1. # Scalable dust fraction where f_dust = 1 has no cleaning and f_dust = 0 has 100% dust cleaning efficiency.

## Noise model [see table 1 of Philcox+2018 for full descriptions]
experiment_profiles= ['Zero', 'BICEP2', 'Simons', 'S4'] # Loaded experimental profiles
experiment_profile_index = 0
# This loads the relevant FWHM, noise_power and delensing_fractions

# These can be overwritten e.g.
a.FWHM = 30. # Experimental noise FWHM [theta_FWHM] in arcmin
a.noise_power = 5. # Experimental noise_power [delta_P] in uK-arcmin
a.delensing_fraction = 1. # Delensing efficiency, where 0.1 implies C_l_lens is reduced by 90%.

## Null testing parameters [paper figure 2]
a.f_dust_all = list(np.arange(1.0,-0.05,-0.05)) # List of values of dust fraction to be tested.
a.err_repeats = 10 # Number of times to repeat each data-point (for error-bars)

## Noise parameter space studies [paper figure 4]
a.noi_par_NoisePower=np.linspace(0.1,5.1,20) # noise_power ranges
a.noi_par_FWHM=np.linspace(0,31.,20) # noise FWHM values
a.noi_par_delensing=[0.1,1.0] # delensing values


## 2.2 Analysis of a Single Map

We here describe how to run the dust analysis for a given HEALPix map, adding the relevant levels of noise, FWHM and lensing as described above. First the `hades/params.py` file must be configured as described above.

To run a single analysis use the wrapper code as follows:

Main code structure: